## Initializing opencv haarcascade face detection network
https://github.com/opencv/opencv/tree/master/data/haarcascades
explained in: https://www.youtube.com/watch?v=7IFhsbfby9s&t=300s (or gitHub)

In [1]:
import cv2
import os

In [2]:
def convertImg(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def faceNetLocalize(img, **kwargs):
    scaleFactor = kwargs.get('scaleFactor', 1.1) #between 1.05 (quality) and 1.4 (speed) recommended (scale of the faces we search for)
    minNeighbors = kwargs.get('minNeighbors', 4) #between 3 (quantity) and 6 (quality) recommended
    minSize = kwargs.get('minSize', (10, 10)) #min size of a face in the picture
    faceNet = kwargs.get('faceNet', init_faceNet())
    
    img_cvt = convertImg(img)
    return faceNet.detectMultiScale3(img_cvt, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize, outputRejectLevels = True)

def init_faceNet(**kwargs):
    path = kwargs.get('path', 'haarcascade_frontalface_default.xml')
    return cv2.CascadeClassifier(path)


In [3]:
#TODO: testing with different model types
#for example: eye model + larger bounding box towards the bottom

## Mask classifier

foundation: https://www.tensorflow.org/tutorials/load_data/images

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPool2D, Conv2D, Input, Dense, Flatten, AveragePooling2D, Dropout
import tensorflow.keras.layers as lays
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomContrast, RandomFlip, RandomRotation
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential, losses as lfs
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils

#### Data augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

In [5]:
augmentation = Sequential([
  RandomFlip("horizontal"),
  RandomRotation(0.4),
  RandomContrast(0.5)
])

##### variables:

In [6]:
batch_size = 32
img_size = (180, 180)
img_size_vgg = (224, 224)
epochs = 11
checkpoint_path = "mask_model/weights.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
imgs_path = os.path.join('..', 'img')
num_classes = 3

correct_usage =  'correct usage: \n' + 'predict([path to image], \'category\' \n' + 'predict([path to image], \'probabilities\' \n' + 'predict([path to image], \'detection\' \n' + 'predict(\'live_detection\')'

##### loading dataset:

In [7]:
def load_dataset(**kwargs):
    imgs_path = kwargs.get('imgs_path', os.path.join('..', 'img'))
    img_size = kwargs.get('img_size', (180, 180))
    batch_size = kwargs.get('batch_size', 32)

    ##load images/ labels from directory
    valid_images = [".jpg",".png",".jpeg",".JPG"]
    x=[]
    y=[]
    
    for root, dirs, files in os.walk(imgs_path):
        for filename in files:
            end = os.path.splitext(filename)[1]
            if end.lower() not in valid_images:
                continue
            image = load_img(os.path.join(root, filename), target_size=img_size)
            image = img_to_array(image)
            
            label = os.path.join(root, filename).split(os.path.sep)[-2]
            
            x.append(image)
            y.append(label)
    x = np.array(x, dtype="float32")
    y = np.array(y)

    ## convert labels to ints from 0 ... len(labels)-1
    labels = []
    for i in range(len(y)):
        try:
            j = labels.index(y[i])
        except:
            labels.append(y[i])
            j = labels.index(y[i])
        y[i] = j
    y.astype(int)
    
    ## split dataset
    trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.2, random_state=3)

    ## one-hot encoding
    trainY = utils.to_categorical(trainY, num_classes)
    testY = utils.to_categorical(testY, num_classes)

    ## data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    ## merge xs and ys
    train_batches = datagen.flow(trainX, trainY, batch_size=32, subset='training')
    test_batches  = datagen.flow(trainX, trainY, batch_size=32, subset='validation')
    
    return train_batches, test_batches, labels, testX, testY, trainX, trainY

### Model selection & Training

In [8]:
#Hypermodels
#https://www.analyticsvidhya.com/blog/2021/06/create-convolutional-neural-network-model-and-optimize-using-keras-tuner-deep-learning/
#https://www.tensorflow.org/tutorials/keras/keras_tuner

def basic_model_builder(hp):
  
    basic_model = Sequential([
        augmentation,
        Conv2D(filters=hp.Int('c1_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c1_kernel', values=[3,5]), activation='relu'),
        AveragePooling2D(pool_size=(7,7)),
        Flatten(name="flatten"),
        Dense(units=hp.Int('d1_units', min_value=32, max_value=512, step=32), activation="relu"),
        Dropout(0.2),#drops small confidences
        Dense(num_classes, activation="softmax")
    ])

    basic_model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=lfs.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return basic_model

def small_model_builder(hp):
  
    small_model = Sequential([
        augmentation,
        Conv2D(filters=hp.Int('c1_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c1_kernel', values=[3,5]), activation='relu'),
        Conv2D(filters=hp.Int('c2_filter', min_value=32, max_value=256, step=16), kernel_size=hp.Choice('c2_kernel', values=[3,5]), activation='relu'),
        MaxPool2D(pool_size=(3,3)),
        Flatten(name="flatten"),
        Dense(units=hp.Int('d1_units', min_value=32, max_value=512, step=32), activation="relu"),
        Dropout(0.2),#drops small confidences
        Dense(num_classes, activation="softmax")
    ])

    small_model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])), loss=lfs.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return small_model


def tune_model(model_builder, xs, ys):
    #tuner = kt.RandomSearch(model_builder, objective='val_accuracy', max_trials=5)
    tuner = kt.RandomSearch(kt.applications.HyperResNet(input_shape=(180, 180, 3), classes=2), objective='val_loss', max_trials=5)
    tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10, factor=3)
    tuner.search(xs, ys, epochs=50, validation_split=0.2)
    return tuner.get_best_hyperparameters(num_trials=1)[0], tuner

In [9]:
def select_model(model_name, **kwargs):
    num_labels = kwargs.get('num_classes', num_classes)

    basic_model = Sequential()
    basic_model.add(Rescaling(1. /255))
    basic_model.add(Conv2D(32, (3,3), activation='relu', input_shape=(180,180,3)))
    basic_model.add(AveragePooling2D(pool_size=(7,7)))
    basic_model.add(Flatten(name="flatten"))
    basic_model.add(Dense(128, activation="relu"))
    basic_model.add(Dropout(0.5)) #drops small confidences
    basic_model.add(Dense(num_labels, activation="softmax"))
    

    small_model = Sequential()
    small_model.add(Rescaling(1. /255))
    small_model.add(Conv2D(filters=128, kernel_size=(5,5), activation='relu', input_shape=(180,180,3)))
    small_model.add(Conv2D(filters=128, kernel_size=(5,5), activation='relu'))
    small_model.add(MaxPool2D(pool_size=(3,3)))
    small_model.add(Flatten(name="flatten"))
    small_model.add(Dense(units=224, activation="relu"))
    small_model.add(Dropout(0.5))#drops small confidences
    small_model.add(Dense(num_labels, activation="softmax"))
    

    vgg_small_model=Sequential()
    
    vgg_small_model.add(Conv2D(64,(3,3),activation='relu',input_shape=(180,180,3)))
    vgg_small_model.add(MaxPool2D(2,2))
    vgg_small_model.add(Conv2D(64,(3,3),activation='relu'))
    vgg_small_model.add(MaxPool2D(2,2))
    vgg_small_model.add(Conv2D(128,(3,3),activation='relu'))
    vgg_small_model.add(MaxPool2D(2,2))
    vgg_small_model.add(Conv2D(128,(3,3),activation='relu'))
    vgg_small_model.add(MaxPool2D(2,2))
    vgg_small_model.add(Flatten())
    vgg_small_model.add(Dropout(0.5))
    vgg_small_model.add(Dense(120,activation='relu'))
    vgg_small_model.add(Dense(3,activation='softmax'))

    vgg_smaller_model=Sequential()
    
    vgg_smaller_model.add(Conv2D(64,(3,3),activation='relu',input_shape=(180,180,3)))
    vgg_smaller_model.add(MaxPool2D(2,2))
    vgg_smaller_model.add(Conv2D(128,(3,3),activation='relu'))
    vgg_smaller_model.add(MaxPool2D(2,2))
    vgg_smaller_model.add(Flatten())
    vgg_smaller_model.add(Dropout(0.5))
    vgg_smaller_model.add(Dense(120,activation='relu'))
    vgg_smaller_model.add(Dense(3,activation='softmax'))

    vgg_model = Sequential()
    vgg_model.add(Rescaling(1. /255))
    vgg_model.add(Conv2D(input_shape=(224,224,3), filters=64, kernel_size=(3,3), padding="same", activation="relu", strides=(1,1))) 
    vgg_model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    vgg_model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    vgg_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(MaxPool2D(pool_size=(2, 2), strides=(2)))
    vgg_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(MaxPool2D(pool_size=(2, 2), strides=(2)))
    vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(MaxPool2D(pool_size=(2, 2), strides=(2)))
    vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    vgg_model.add(MaxPool2D(pool_size=(2, 2), strides=(2)))
    vgg_model.add(Flatten())
    vgg_model.add(Dense(units=4096, activation="relu"))
    vgg_model.add(Dense(units=4096, activation="relu"))
    vgg_model.add(Dense(units=num_labels, activation="softmax"))
    

    if model_name == 'basic_model':
        #basic_model.summary()
        return basic_model
    
    if model_name == 'small_model':
        #small_model.summary()
        return small_model

    if model_name == 'vgg_model':
        #vgg_model.summary()
        return vgg_model
    
    if model_name == 'vgg_small_model':
        #vgg_small_model.summary()
        return vgg_small_model

about the VGG-model: https://neurohive.io/en/popular-networks/vgg16/

In [10]:
def train_model(model, train_ds, val_ds, **kwargs):
    epochs = kwargs.get('epochs', 10)

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    return history

#### Testing/Evaluation (mask):
https://www.tensorflow.org/guide/keras/train_and_evaluate

In [11]:
def evaluate_model(x_test, y_test, model):
    results = model.evaluate(x_test, y_test, batch_size=32)
    print(results)
    
    y_pred_confidences = model.predict(x_test)
    y_pred = [np.argmax(cs) for cs in y_pred_confidences]
    print(classification_report(y_test, y_pred))

##### Predictions:

In [12]:
def load_model(**kwargs):
    checkpoint_path = kwargs.get('checkpoint_path', "mask_model/weights.ckpt")
    model = kwargs.get('model', select_model('basic_model'))
    model.load_weights(checkpoint_path)
    return model

In [34]:
def maskPredict(model, img, labels):
    pred = model.predict(img[None])
    label_index = np.argmax(pred)
    print(labels[label_index])
    return labels[label_index], pred[0][label_index]


#mode can be 'category', 'probabilities', 'detection', 'live_detection'
def predict(mode, model, **kwargs):
    #model = kwargs.get('model', load_model())
    img_path = kwargs.get('img_path', None)
    img_size = kwargs.get('img_size', (180, 180))
    labels = kwargs.get('labels', None)
    
    
    if mode=='live_detection':
        live_det(model, img_size, labels)
        return
    
    if img_path is None:
        print(correct_usage)
        return

    #load_img
    img = load_img(img_path, target_size = img_size)
    img = img_to_array(img)
    
    if mode=='detection':
        detect(img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return
        
    if mode=='category':
        label, confidence = maskPredict(model, img, labels)
        return label
        
    if mode=='probabilities':
        #TODO: schöneres Format
        return model.predict(img[None])

    else:
        print(correct_usage)

def detect(model, img, img_size, labels):
    faceLocs, rejectLevels, confidences = faceNetLocalize(img)
        
    for (x, y, w, h) in faceLocs:
        #crop image and predict label of cropped image
        img_crop = img[y:y+w, x:x+h]
        img_crop = cv2.resize(img_crop, img_size)
        img_crop = img_crop / 255
        label, confidence_mask = maskPredict(model, img_to_array(img_crop), labels)

        #show label/ bounding box on image
        cv2.putText(img, f"{label}, confidence:{confidence_mask}", (x, y), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2) 
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
    
    cv2.imshow('live_output', img)

def live_det(model, img_size, labels):
    wait_time = 10 #time in ms to wait before refreshing feed
    camera = cv2.VideoCapture(0) #Input value might differ on different systems
    
    while(True):
        ret, img = camera.read()
        if not ret:
            print('Error: failed reading camera')
            return 'Error: failed reading camera'
        detect(model, img, img_size, labels)

        #wait for ESC or q
        if (cv2.waitKey(wait_time) & 0xFF) in [27, ord('q')]: 
            break

    camera.release()
    return 'live_output'



In [14]:
train_ds, test_ds, labels, testX, testY, trainX, trainY = load_dataset()
#bhps, tuner = tune_model(basic_model_builder, x_test, y_test)

In [15]:
model = select_model('basic_model')
#model = tuner.hypermodel.build(bhps)
history = train_model(model, train_ds, test_ds, epochs=20)

#evaluate_model(testX, testY, model)

C:\Users\Julie\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\Julie\anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/20
8/8 [==============================] - 5s 586ms/step - loss: 2.3970 - accuracy: 0.2571 - val_loss: 0.9822 - val_accuracy: 0.4754
Epoch 2/20
8/8 [==============================] - 4s 548ms/step - loss: 1.0546 - accuracy: 0.4367 - val_loss: 0.9217 - val_accuracy: 0.6721
Epoch 3/20
8/8 [==============================] - 4s 523ms/step - loss: 0.9066 - accuracy: 0.6286 - val_loss: 0.8746 - val_accuracy: 0.6393
Epoch 4/20
8/8 [==============================] - 4s 512ms/step - loss: 0.8395 - accuracy: 0.6490 - val_loss: 0.7718 - val_accuracy: 0.7213
Epoch 5/20
8/8 [==============================] - 4s 515ms/step - loss: 0.7730 - accuracy: 0.6408 - val_loss: 0.7504 - val_accuracy: 0.6230
Epoch 6/20
8/8 [==============================] - 4s 512ms/step - loss: 0.7301 - accuracy: 0.7265 - val_loss: 0.6239 - val_accuracy: 0.8361
Epoch 7/20
8/8 [==============================] - 4s 509ms/step - loss: 0.6949 - accuracy: 0.7102 - val_loss: 0.5758 - val_accuracy: 0.8197
Epoch 8/20
8/8 [====

In [16]:
print("loss =",history.history["loss"])
print("val_loss =",history.history["val_loss"])
print("acc =",history.history["accuracy"])
print("val_acc =",history.history["val_accuracy"])

loss = [2.3970251083374023, 1.0545551776885986, 0.9066411852836609, 0.8395401239395142, 0.7729923725128174, 0.730128824710846, 0.6948890686035156, 0.6645703315734863, 0.6187208294868469, 0.5959728360176086, 0.60105961561203, 0.6316047310829163, 0.5675155520439148, 0.49010926485061646, 0.5262033939361572, 0.5557922124862671, 0.4815269708633423, 0.5259525179862976, 0.5021576285362244, 0.4851398468017578]
val_loss = [0.9821628332138062, 0.921679675579071, 0.8746117949485779, 0.7717764377593994, 0.7503803372383118, 0.6238858699798584, 0.5757790207862854, 0.5477417707443237, 0.5252807140350342, 0.5514279007911682, 0.5235510468482971, 0.4636566936969757, 0.4903154671192169, 0.4438585638999939, 0.4100685119628906, 0.41123974323272705, 0.4665105640888214, 0.35066360235214233, 0.4116128087043762, 0.5270665884017944]
acc = [0.2571428716182709, 0.43673470616340637, 0.6285714507102966, 0.6489796042442322, 0.640816330909729, 0.7265306115150452, 0.7102040648460388, 0.7020407915115356, 0.730612218379

In [37]:
img_path='../img/ffp2/IMG_1599.JPG_(350.0, 92.0, 1009.0, 599.0).png'
#img_path='../img/no_mask/IMG_1323.JPG_(1025, 427, 984, 984).png'
#img_path='../img/op_mask/IMG_1337.JPG_(878, 710, 954, 954).png'
#img_path='../img/other_mask/IMG_1327.JPG_(863, 476, 1097, 1097).png'
predict('category',img_path=img_path,  model=model, labels=labels, img_size=(180,180))

#predict('live_detection', model=model, labels=labels, img_size=(180,180))

op_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
no_mask
